### 画海表面温度sst的时间序列(包含年均和月数据)
#### use NCL(6.6.2)

In [1]:
output                 = "/home/yyq/mls/dataprocess/warming/photo/timeseries"
datapath               = "/data06/yyq/mls/licom2/GM-Case1501/exe/"

In [2]:
fileslist              = systemfunc("ls "+ datapath+"MMEAN" +"*") 
f_year                 = addfile(datapath+"YME.nc", "r")
f                      = addfiles(fileslist, "r") 

In [3]:
sst_ori                = f[:]->ts(:,0,:,:)
sst_year_ori           = f_year->ts(:,0,:,:)
printVarSummary(sst_year_ori)

Variable: sst_year_ori
Type: float
Total Size: 5106920 bytes
            1276730 values
Number of Dimensions: 3
Dimensions and sizes:	[time | 61] x [lat | 115] x [lon | 182]
Coordinates: 
            time: [   1.. 721]
            lat: [90..-78]
            lon: [ 0..362]
Number Of Attributes: 5
  lev :	-5
  long_name :	temperature
  units :	centigrade
  _FillValue :	1e+35
  missing_value :	1e+35

In [4]:
lon_size               = dimsizes(sst_year_ori(0,0,:)) - 1 - 2
time_size              = dimsizes(sst_year_ori(:,0,0)) - 1

In [5]:
sst                    = sst_ori(:,:,0:lon_size)
sst_year               = sst_year_ori(:,:,0:lon_size)

In [6]:
lat                    = f_year ->lat(:)
angle2radian           = 4.0 * atan(1.0) / 180.0
dlat                   = cos (lat*angle2radian)

In [7]:
sst_area               = wgt_areaave_Wrap(sst, dlat, 1.0, 1)
sst_year_area          = wgt_areaave_Wrap(sst_year, dlat, 1.0, 1)
sst_year_area          = runave(sst_year_area, 9, 0)
printVarSummary(sst_area)
printVarSummary(sst_year_area)

Variable: sst_area
Type: float
Total Size: 2928 bytes
            732 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 732]
Coordinates: 
            time: [   1.. 732]
Number Of Attributes: 5
  _FillValue :	1e+35
  units :	centigrade
  long_name :	temperature
  lev :	-5
  wgt_areaave_op_ncl :	Area AverageVariable: sst_year_area
Type: float
Total Size: 244 bytes
            61 values
Number of Dimensions: 1
Dimensions and sizes:	[time | 61]
Coordinates: 
            time: [   1.. 721]
Number Of Attributes: 6
  _FillValue :	1e+35
  units :	centigrade
  long_name :	temperature
  lev :	-5
  missing_value :	1e+35
  wgt_areaave_op_ncl :	Area Average

In [8]:
sst_end = dim_avg_n_Wrap(sst_year((time_size - 2):time_size,:,:), 0)
sst_start = dim_avg_n_Wrap(sst_year(0:2,:,:), 0)
sst_diff = sst_end - sst_start
sst_diff_lat = dim_avg_n_Wrap(sst_diff, 1)
sst_diff_lat = runave(sst_diff_lat, 9, 0)
copy_VarMeta(sst_end(:,0), sst_diff_lat)
printVarSummary(sst_diff_lat)

Variable: sst_diff_lat
Type: float
Total Size: 460 bytes
            115 values
Number of Dimensions: 1
Dimensions and sizes:	[lat | 115]
Coordinates: 
            lat: [90..-78]
Number Of Attributes: 7
  missing_value :	1e+35
  lev :	-5
  long_name :	temperature
  units :	centigrade
  average_op_ncl :	dim_avg_n over dimension(s): time
  lon :	 0
  _FillValue :	1e+35

In [9]:
month                  = dimsizes(sst_area)
x                      = new((/month/),float)  
x                      = ispan(1,month,1)
x                      = x / 12
year                   = dimsizes(sst_year_area)
xx                     = new((/year/),float) 
xx                     = ispan(1,year,1)

In [27]:
wks      = gsn_open_wks("png", output)
plots    = new(2,"graphic")
res_0    = True
    res_0@gsnDraw                = False
    res_0@gsnFrame               = False
    res_0@tiXAxisString          = "year" 
    res_0@tiYAxisString          = "SST"
    res_0@gsnRightString          = "K"
    res_0@trXMinF                = 0.0
    res_0@trXMaxF                = 60.0
    res_0@vpHeightF              = 0.37
    res_0@vpWidthF               = 0.6
res_overlay = True
    res_overlay@gsnDraw                = False
    res_overlay@gsnFrame               = False
    res_overlay@xyLineColor      = "red"
    res_overlay@xyLineThicknesses = 2.5
plots(0) = gsn_csm_xy(wks, x, sst_area, res_0)
plot_overlay = gsn_csm_xy(wks, xx, sst_year_area, res_overlay)
overlay(plots(0), plot_overlay)

res_diff = True
    res_diff@gsnDraw                = False
    res_diff@gsnFrame               = False
    res_diff@tiXAxisString          = "latutude" 
    res_diff@tiYAxisString          = "delta SST"
    res_diff@gsnRightString         = "K"
    res_diff@trXMinF                = -62
    res_diff@trXMaxF                = 62
    res_diff@vpHeightF              = 0.37
    res_diff@vpWidthF               = 0.6
    res_diff@xyLineThicknesses      = 2.5
    res_diff@xyLineColor            = "red"
plots(1) = gsn_csm_xy(wks, lat, sst_diff_lat, res_diff)

resP     = True
gsn_panel(wks, plots, (/1,2/), resP)

![](../photo/timeseries.png)